In [5]:
import os
from pathlib import Path
import pandas as pd
from tabulate import tabulate

In [6]:
os.environ["DATA_DIR"] = str(Path().cwd().parent.parent / "data")

In [7]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.set_option("display.max_colwidth", None)

In [8]:
def read_data(path: Path) -> pd.DataFrame:
    return pd.read_json(path)

In [9]:
def tabulate_data(path: Path):
    output_dir = Path().cwd().parent / "output"
    print(f"tabulating...: {path}")
    df = read_data(path)
    (output_dir / f"{path.stem}.txt").write_text(
        tabulate(df, headers="keys", tablefmt="grid", showindex=False)
    )


# for i in Path(os.getenv("DATA_DIR")).rglob("*.json"):
#     tabulate_data(i)

In [ ]:
df = read_data(f"{os.getenv('DATA_DIR')}/cost_of_living.json")
df.pivot(index="city", columns="item", values="cost")

(715, 702)